# Linear Models

In [1]:
import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get Data

In [2]:
def get_subject_from_path(path):
    normalized_path = os.path.normpath(path)
    path_components = normalized_path.split(os.sep)
    return path_components[-2][4:]

In [38]:
# Search for all functional connectivity files and read them into a
# numpy array.
fc_path = '/imaging3/owenlab/bpho/python_yeo_fc'
fc_paths = glob.glob(fc_path + '/**/yeo_fc.npy', recursive=True)

within_fpn = {}
within_dmn = {}
between_fpn_dmn = {}

fpn_indices = np.triu_indices(11, k=1)
dmn_indices = (np.triu_indices(8, k=1)[0] + 11, np.triu_indices(8, k=1)[1] + 11)

for path in fc_paths:
    subject_id = get_subject_from_path(path)
    subject_fc = np.load(path)
    within_fpn[subject_id] = subject_fc[fpn_indices]
    within_dmn[subject_id] = subject_fc[dmn_indices]
    between_fpn_dmn[subject_id] = subject_fc[:11, 11:]
    
#     plt.figure(figsize=(10, 10))
#     plt.imshow(subject_fc)
#     a = np.zeros((11, 11))
#     a[fpn_indices] = subject_fc[fpn_indices]
#     plt.imshow(a)
#     b = np.zeros((8, 8))
#     b[np.triu_indices(8, k=1)] = subject_fc[dmn_indices]
#     plt.imshow(b)
#     plt.imshow(subject_fc[:11, 11:])

print("Number of functional connectivities:", len(fcs))
print("Number of within-FPN connections:", within_fpn["NDARAP912JK3"].shape)
print("Number of within-DMN connections:", within_dmn["NDARAP912JK3"].shape)
print("Number of between-FPN-DMN connections:", between_fpn_dmn["NDARAP912JK3"].shape)

Number of functional connectivities: 722
Number of within-FPN connections: (55,)
Number of within-DMN connections: (28,)
Number of between-FPN-DMN connections: (11, 8)


## Get Labels

In [6]:
wisc_label_path = "/imaging3/owenlab/bpho/Biobank Labels/Subjects_with_WISC.csv"
wisc_labels = pd.read_csv(wisc_label_path, index_col='assessment WISC,EID')
# display(wisc_labels)

In [7]:
subject_ids = wisc_labels.index
measure_text = "WISC_FSIQ"
measure_col = f'assessment WISC,{measure_text}'
# measure_text = "Age"
# measure_col = 'assessment Basic_Demos,Age'

fc_matrices = []
measures = []
ages = []

for subject_id in subject_ids:
    if subject_id not in fcs:
        continue
    
    fc_matrices.append(fcs[subject_id])
    measures.append(wisc_labels.at[subject_id, measure_col])
    ages.append(wisc_labels.at[subject_id, 'assessment Basic_Demos,Age'])
    
print("Num subjects with measures:", len(measures))

Num subjects with measures: 577


In [8]:
X = np.array(fc_matrices)
y = np.array(measures)

print("X shape:", X.shape, "y shape:", y.shape)

X shape: (577, 171) y shape: (577,)


## Linear Regression

In [9]:
from scipy import stats

In [10]:

slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 171 and the array at index 1 has size 577